In [201]:
import numpy as np
import pandas as pd
import scipy.stats as st
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [528]:
df = pd.read_csv('spreadspoke_scores.csv')
df.dtypes

schedule_season          int64
schedule_week            int64
team_home               object
team_away               object
team_favorite_id        object
spread_favorite        float64
over_under_line        float64
weather_detail          object
weather_temperature    float64
weather_wind_mph       float64
weather_humidity       float64
score_home               int64
score_away               int64
point_total              int64
over_under_result       object
score_difference         int64
home_id                 object
away_id                 object
home_favorite            int64
favorite_covered        object
winning_team            object
losing_team             object
home_wins                int64
home_losses              int64
home_ties                int64
away_wins                int64
away_losses              int64
away_ties                int64
dtype: object

In [348]:
df = df.dropna(subset=['team_favorite_id', 'spread_favorite', 'over_under_line'])
df.index= range(0,len(df))
df = df.drop(columns=['game_id','stadium_neutral','schedule_playoff', 'stadium','schedule_date'])

In [349]:
df['weather_humidity']=df['weather_humidity'].apply(pd.to_numeric, errors='coerce')
df['over_under_line']=df['over_under_line'].apply(pd.to_numeric, errors='coerce')
#df['schedule_date']=df['schedule_date'].apply(pd.to_datetime)

In [350]:
#cleaning team_fav_id to correctly match team names in instances where team moved city
df.loc[df.team_favorite_id =='SD','team_favorite_id'] = 'LAC'
df.loc[(df.team_favorite_id=='LAR') & (df.team_home!= 'Los Angeles Rams') & (df.team_away!= 'Los Angeles Rams') & (df.team_home!= 'St. Louis Rams') & (df.team_away!= 'St. Louis Rams'), 'team_favorite_id']='OAK'
df.loc[((df.team_home=='Baltimore Colts') | (df.team_away=='Baltimore Colts')) & (df.team_favorite_id=='BAL'), 'team_favorite_id'] = 'IND'

In [351]:
special_weather_df = df[(df.weather_detail.isna() == False) & (df.weather_detail != 'DOME')]

In [352]:
#convert playoff rounds into ints (started at 19 bc seasons '93 and '99 had 18 weeks)
df.loc[(df.schedule_week=='WildCard') | (df.schedule_week=='Wildcard'), 'schedule_week']=19
df.loc[(df.schedule_week=='Division'), 'schedule_week']=20
df.loc[(df.schedule_week=='Conference'), 'schedule_week']=21
df.loc[(df.schedule_week=='Superbowl') | (df.schedule_week=='SuperBowl'), 'schedule_week']=22

#convert into ints
df.schedule_week = df.schedule_week.astype('int64')

In [353]:
df['point_total'] = df['score_home'] + df['score_away']

In [354]:
df['over_under_result'] = np.where(df['point_total']>df['over_under_line'], 'over', 'under')
df['over_under_result'] = np.where(df['point_total']==df['over_under_line'], 'push', df['over_under_result'])

In [355]:
df['score_difference'] = df['score_home'] - df['score_away']

In [356]:
teams_dict = {'Baltimore Ravens': 'BAL', 'Jacksonville Jaguars': 'JAX',
 'Carolina Panthers': 'CAR', 'Green Bay Packers': 'GB','St. Louis Cardinals': 'ARI',
 'Los Angeles Raiders': 'OAK','Detroit Lions': 'DET','Cincinnati Bengals': 'CIN',
 'Los Angeles Rams': 'LAR','Oakland Raiders': 'OAK',
 'Houston Texans': 'HOU', 'Tennessee Titans': 'TEN', 'New York Jets': 'NYJ',
 'St. Louis Rams': 'LAR', 'Baltimore Colts': 'IND', 'Los Angeles Chargers': 'LAC',
 'Phoenix Cardinals': 'ARI', 'San Diego Chargers': 'LAC', 'Indianapolis Colts': 'IND',
 'Washington Redskins': 'WAS', 'Miami Dolphins': 'MIA', 'Buffalo Bills': 'BUF',
 'Tennessee Oilers': 'TEN', 'Minnesota Vikings': 'MIN', 'Atlanta Falcons': 'ATL',
 'New Orleans Saints': 'NO', 'Pittsburgh Steelers': 'PIT','Kansas City Chiefs': 'KC', 
 'Denver Broncos': 'DEN', 'New York Giants': 'NYG', 'Houston Oilers': 'TEN',
 'Cleveland Browns': 'CLE', 'Philadelphia Eagles': 'PHI', 'Dallas Cowboys': 'DAL',
 'Arizona Cardinals': 'ARI', 'Chicago Bears': 'CHI', 'New England Patriots': 'NE',
 'San Francisco 49ers': 'SF','Tampa Bay Buccaneers': 'TB','Seattle Seahawks': 'SEA'}
teams_dict

{'Baltimore Ravens': 'BAL',
 'Jacksonville Jaguars': 'JAX',
 'Carolina Panthers': 'CAR',
 'Green Bay Packers': 'GB',
 'St. Louis Cardinals': 'ARI',
 'Los Angeles Raiders': 'OAK',
 'Detroit Lions': 'DET',
 'Cincinnati Bengals': 'CIN',
 'Los Angeles Rams': 'LAR',
 'Oakland Raiders': 'OAK',
 'Houston Texans': 'HOU',
 'Tennessee Titans': 'TEN',
 'New York Jets': 'NYJ',
 'St. Louis Rams': 'LAR',
 'Baltimore Colts': 'IND',
 'Los Angeles Chargers': 'LAC',
 'Phoenix Cardinals': 'ARI',
 'San Diego Chargers': 'LAC',
 'Indianapolis Colts': 'IND',
 'Washington Redskins': 'WAS',
 'Miami Dolphins': 'MIA',
 'Buffalo Bills': 'BUF',
 'Tennessee Oilers': 'TEN',
 'Minnesota Vikings': 'MIN',
 'Atlanta Falcons': 'ATL',
 'New Orleans Saints': 'NO',
 'Pittsburgh Steelers': 'PIT',
 'Kansas City Chiefs': 'KC',
 'Denver Broncos': 'DEN',
 'New York Giants': 'NYG',
 'Houston Oilers': 'TEN',
 'Cleveland Browns': 'CLE',
 'Philadelphia Eagles': 'PHI',
 'Dallas Cowboys': 'DAL',
 'Arizona Cardinals': 'ARI',
 'Chicago 

In [357]:
#create home and away id columns
df['home_id'] = ''
df['away_id'] = ''
df['home_id'] = df.team_home.map(teams_dict)
df['away_id'] = df.team_away.map(teams_dict)


In [358]:
df['home_favorite'] = 0
df.loc[df.home_id == df.team_favorite_id,'home_favorite'] = 1
df

,schedule_season,schedule_week,team_home,team_away,team_favorite_id,spread_favorite,over_under_line,weather_detail,weather_temperature,weather_wind_mph,weather_humidity,score_home,score_away,point_total,over_under_result,score_difference,home_id,away_id,home_favorite
350,1967,22,Green Bay Packers,Oakland Raiders,GB,-13.5,43.0,60 degrees- humidity 74%- wind 12 mph,60.0,12.0,74.0,33,14,47,over,19,GB,OAK,1
538,1968,22,Baltimore Colts,New York Jets,IND,-18.0,40.0,66 degrees- humidity 80%- wind 12 mph,66.0,12.0,80.0,7,16,23,under,-9,IND,NYJ,1
727,1969,22,Kansas City Chiefs,Minnesota Vikings,MIN,-12.0,39.0,55 degrees- humidity 84%- wind 14 mph,55.0,14.0,84.0,23,7,30,under,16,KC,MIN,0
916,1970,22,Baltimore Colts,Dallas Cowboys,IND,-2.5,36.0,59 degrees- humidity 60%- wind 11 mph,59.0,11.0,60.0,16,13,29,under,3,IND,DAL,1
1105,1971,22,Dallas Cowboys,Miami Dolphins,DAL,-6.0,34.0,34 degrees- humidity 40%- wind 18 mph- wind c...,34.0,18.0,40.0,24,3,27,under,21,DAL,MIA,1
1294,1972,22,Miami Dolphins,Washington Redskins,MIA,-1.0,33.0,64 degrees- humidity 53%- wind 7 mph,64.0,7.0,53.0,14,7,21,under,7,MIA,WAS,1
1483,1973,22,Miami Dolphins,Minnesota Vikings,MIA,-6.5,33.0,47 degrees- humidity 81%- wind 7 mph,47.0,7.0,81.0,24,7,31,under,17,MIA,MIN,1
1672,1974,22,Minnesota Vikings,Pittsburgh Steelers,PIT,-3.0,33.0,51 degrees- humidity 72%- wind 17 mph,51.0,17.0,72.0,6,16,22,under,-10,MIN,PIT,0
1861,1975,22,Dallas Cowboys,Pittsburgh Steelers,PIT,-7.0,36.0,49 degrees- humidity 44%- wind 18 mph,49.0,18.0,44.0,17,21,38,over,-4,DAL,PIT,0
2064,1976,22,Minnesota Vikings,Oakland Raiders,OAK,-4.0,38.0,52 degrees- humidity 59%- wind 6 mph,52.0,6.0,59.0,14,32,46,over,-18,MIN,OAK,0


In [359]:
#determine whether favorite covered the spread, or push if even
df['favorite_covered'] = np.where((df.home_favorite==1) & (np.abs(df.score_difference) > np.abs(df.spread_favorite)),1,0)
df['favorite_covered'] = np.where((np.abs(df.score_difference) == np.abs(df.spread_favorite)),'2',df.home_favorite)


In [360]:
#which team won the game outright (no spread)
df['winning_team'] = ''
df['winning_team'] = np.where(df.score_home > df.score_away, df.home_id, df.away_id)
df['winning_team'] = np.where(df.score_home == df.score_away, 'tie', df.winning_team)

df['losing_team'] = ''
df['losing_team'] = np.where(df.score_home < df.score_away, df.home_id, df.away_id)
df['losing_team'] = np.where(df.score_home == df.score_away, 'tie', df.losing_team)

In [361]:
df['home_wins'] = 0
df['home_losses'] = 0
df['home_ties'] = 0
df['away_wins'] = 0
df['away_losses'] = 0
df['away_ties'] = 0

In [345]:
sum((df['schedule_season'] == 2017) & (df['schedule_week'] < 5) & (df['winning_team'] == 'NE'))

2

In [414]:
def get_record(df):
    home_wins = []
    home_losses = []
    home_ties = []
    away_wins = []
    away_losses = []
    away_ties = []
    for index, row in df.iterrows():
        week = row.schedule_week
        season = row.schedule_season
        home_id = row.home_id
        away_id = row.away_id
        
        hw = sum((df['schedule_season'] == season) & (df['schedule_week'] < week) & (df['winning_team'] == home_id))
        home_wins.append(hw)
        hl = sum((df['schedule_season'] == season) & (df['schedule_week'] < week) & (df['losing_team'] == home_id))
        home_losses.append(hl)
        ht = sum((df['schedule_season'] == season) & (df['schedule_week'] < week) & (df['winning_team'] == 'tie') & (df['home_id'] == home_id))
        home_ties.append(ht)
        
        aw = sum((df['schedule_season'] == season) & (df['schedule_week'] < week) & (df['winning_team'] == away_id))
        away_wins.append(aw) 
        al = sum((df['schedule_season'] == season) & (df['schedule_week'] < week) & (df['losing_team'] == away_id))
        away_losses.append(al)
        at = sum((df['schedule_season'] == season) & (df['schedule_week'] < week) & (df['winning_team'] == 'tie') & (df['away_id'] == away_id))
        away_ties.append(at)
        
    return home_wins, home_losses, home_ties, away_wins, away_losses, away_ties
        
home_wins, home_losses, home_ties, away_wins, away_losses, away_ties = get_record(df)

In [280]:
df_filter = df.filter(items=['schedule_season', 'schedule_week', 'winning_team', 'home_id','home_wins'])

In [426]:
df['home_wins'] = home_wins
df['home_losses'] = home_losses
df['home_ties'] = home_ties
df['away_wins'] = away_wins
df['away_losses'] = away_losses
df['away_ties'] = away_ties

#df[(df.schedule_season==2017) & ((df.home_id == 'NYG') | (df.away_id == 'NYG'))]

In [492]:
def get_point_differential(df):
    result_array = np.empty((0,4))
    
    for index, row in df.iterrows():
        week = row.schedule_week
        season = row.schedule_season
        team_1 = row.home_id
        team_2 = row.away_id
        
        team_1_home_points=np.sum(df.score_home[(df['schedule_season'] == season) & (df['schedule_week'] < week) & (df['home_id'] == team_1)])
        team_1_away_points=np.sum(df.score_away[(df['schedule_season'] == season) & (df['schedule_week'] < week) & (df['away_id'] == team_1)])
        team_1_total_points = team_1_home_points+team_1_away_points
        
        team_1_home_points_against=np.sum(df.score_away[(df['schedule_season'] == season) & (df['schedule_week'] < week) & (df['home_id'] == team_1)])
        team_1_away_points_against=np.sum(df.score_home[(df['schedule_season'] == season) & (df['schedule_week'] < week) & (df['away_id'] == team_1)])
        team_1_total_points_against = team_1_home_points_against+team_1_away_points_against
        
        team_2_home_points=np.sum(df.score_home[(df['schedule_season'] == season) & (df['schedule_week'] < week) & (df['home_id'] == team_2)])
        team_2_away_points=np.sum(df.score_away[(df['schedule_season'] == season) & (df['schedule_week'] < week) & (df['away_id'] == team_2)])
        team_2_total_points = team_2_home_points+team_2_away_points
        
        team_2_home_points_against=np.sum(df.score_away[(df['schedule_season'] == season) & (df['schedule_week'] < week) & (df['home_id'] == team_2)])
        team_2_away_points_against=np.sum(df.score_home[(df['schedule_season'] == season) & (df['schedule_week'] < week) & (df['away_id'] == team_2)])
        team_2_total_points_against = team_2_home_points_against+team_2_away_points_against
    
        result_array = np.append(result_array, values=[[team_1_total_points, team_1_total_points_against, team_2_total_points, team_2_total_points_against]],axis=0)
        
    return result_array
        

In [493]:
point_diff_arr = get_point_differential(df)

In [543]:
# df['home_points_for'] = ''
# df['home_points_against'] = ''
# df['away_points_for'] = ''
# df['away_points_against'] = ''
point_diff_df = pd.DataFrame(point_diff_arr, columns=['home_points_for','home_points_against', 'away_points_for', 'away_points_against'], dtype='int')
point_diff_df

,home_points_for,home_points_against,away_points_for,away_points_against
0,0,0,0,0
1,0,0,0,0
2,0,0,0,0
3,0,0,0,0
4,0,0,0,0
5,0,0,0,0
6,0,0,0,0
7,0,0,0,0
8,0,0,0,0
9,0,0,0,0


In [544]:
point_diff_df.dtypes

home_points_for        int64
home_points_against    int64
away_points_for        int64
away_points_against    int64
dtype: object

In [545]:
df = pd.concat([df,point_diff_df],axis=1)

In [546]:
df

,schedule_season,schedule_week,team_home,team_away,team_favorite_id,spread_favorite,over_under_line,weather_detail,weather_temperature,weather_wind_mph,...,home_wins,home_losses,home_ties,away_wins,away_losses,away_ties,home_points_for,home_points_against,away_points_for,away_points_against
0,1967,22,Green Bay Packers,Oakland Raiders,GB,-13.5,43.0,60 degrees- humidity 74%- wind 12 mph,60.0,12.0,...,0,0,0,0,0,0,0,0,0,0
1,1968,22,Baltimore Colts,New York Jets,IND,-18.0,40.0,66 degrees- humidity 80%- wind 12 mph,66.0,12.0,...,0,0,0,0,0,0,0,0,0,0
2,1969,22,Kansas City Chiefs,Minnesota Vikings,MIN,-12.0,39.0,55 degrees- humidity 84%- wind 14 mph,55.0,14.0,...,0,0,0,0,0,0,0,0,0,0
3,1970,22,Baltimore Colts,Dallas Cowboys,IND,-2.5,36.0,59 degrees- humidity 60%- wind 11 mph,59.0,11.0,...,0,0,0,0,0,0,0,0,0,0
4,1971,22,Dallas Cowboys,Miami Dolphins,DAL,-6.0,34.0,34 degrees- humidity 40%- wind 18 mph- wind c...,34.0,18.0,...,0,0,0,0,0,0,0,0,0,0
5,1972,22,Miami Dolphins,Washington Redskins,MIA,-1.0,33.0,64 degrees- humidity 53%- wind 7 mph,64.0,7.0,...,0,0,0,0,0,0,0,0,0,0
6,1973,22,Miami Dolphins,Minnesota Vikings,MIA,-6.5,33.0,47 degrees- humidity 81%- wind 7 mph,47.0,7.0,...,0,0,0,0,0,0,0,0,0,0
7,1974,22,Minnesota Vikings,Pittsburgh Steelers,PIT,-3.0,33.0,51 degrees- humidity 72%- wind 17 mph,51.0,17.0,...,0,0,0,0,0,0,0,0,0,0
8,1975,22,Dallas Cowboys,Pittsburgh Steelers,PIT,-7.0,36.0,49 degrees- humidity 44%- wind 18 mph,49.0,18.0,...,0,0,0,0,0,0,0,0,0,0
9,1976,22,Minnesota Vikings,Oakland Raiders,OAK,-4.0,38.0,52 degrees- humidity 59%- wind 6 mph,52.0,6.0,...,0,0,0,0,0,0,0,0,0,0
